In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import glob
import re
from pathlib import Path

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Configuration
plots_dir = "playground/plots_gad"  # GAD plot directory
logs_dir = "playground/logs_gad"    # GAD results directory
print(f"Looking for results in: {plots_dir}")
print(f"Looking for logs in: {logs_dir}")


In [ ]:
def find_gad_files(plots_dir, logs_dir):
    """Find all GAD result files in both directories"""
    
    # Find summary.json files from plots directory
    summary_pattern = os.path.join(plots_dir, "**/summary.json")
    summary_files = glob.glob(summary_pattern, recursive=True)
    
    # Find results_*.json files from logs directory  
    results_pattern = os.path.join(logs_dir, "results_*.json")
    results_files = glob.glob(results_pattern, recursive=True)
    
    print(f"Found {len(summary_files)} summary.json files")
    print(f"Found {len(results_files)} results_*.json files")
    
    return summary_files, results_files

def parse_gad_title(title):
    """Parse the GAD run title to extract key information"""
    # Example titles: 
    # "TS from TS qr idx104000"
    # "TS from R svd idx104000" 
    # "TS from R-P interpolation inertia idx104000"
    
    info = {
        'starting_point': None,
        'eigen_method': None,
        'dt': None,
        'max_steps': None,
        'idx': None
    }
    
    # Extract starting point
    if "from TS" in title and "perturbed" not in title:
        info['starting_point'] = "TS"
    elif "from perturbed TS" in title:
        info['starting_point'] = "Perturbed TS"
    elif "from R-P geodesic interpolation" in title:
        info['starting_point'] = "R-P Geodesic"
    elif "from R-P interpolation" in title:
        info['starting_point'] = "R-P Linear"
    elif "from R" in title:
        if "10k steps" in title:
            info['starting_point'] = "R (10k steps)"
        else:
            info['starting_point'] = "R"
    
    # Extract index
    idx_match = re.search(r'idx(\d+)', title)
    if idx_match:
        info['idx'] = int(idx_match.group(1))
    
    # Extract eigen method - look for known methods
    eigen_methods = ["qr", "svd", "svdforce", "inertia", "geo", "ase", "eckartsvd", "eckartqr", "None"]
    for method in eigen_methods:
        if f" {method} " in title or title.endswith(f" {method}"):
            info['eigen_method'] = method
            break
    
    return info


In [ ]:
def parse_results_filename(filename):
    """Parse results_*.json filename to extract eigen method and idx"""
    # Example: "results_qr_idx104000.json"
    basename = os.path.basename(filename)
    
    info = {
        'eigen_method': None,
        'idx': None
    }
    
    # Extract eigen method and idx
    match = re.search(r'results_([^_]+)_idx(\d+)\.json', basename)
    if match:
        info['eigen_method'] = match.group(1)
        info['idx'] = int(match.group(2))
    
    return info

def load_gad_data(summary_files, results_files):
    """Load all GAD data into a comprehensive dataset"""
    data = []
    
    # Load summary files (from trajectory integration)
    for file_path in summary_files:
        try:
            with open(file_path, 'r') as f:
                summary = json.load(f)
            
            # Extract directory name which contains the title
            dir_name = os.path.basename(os.path.dirname(file_path))
            
            # Parse title information
            title_info = parse_gad_title(dir_name)
            
            # Combine summary data with parsed title info
            row = {
                'source': 'summary',
                'file_path': file_path,
                'directory': dir_name,
                **title_info,
                **summary
            }
            
            data.append(row)
            
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    # Load results files (from GAD specific results)
    for file_path in results_files:
        try:
            with open(file_path, 'r') as f:
                results = json.load(f)
            
            # Parse filename info
            file_info = parse_results_filename(file_path)
            
            # Create separate rows for each test scenario
            for test_name, rmsd_value in results.items():
                # Parse test name to extract starting point info
                test_info = {
                    'starting_point': None,
                    'dt': None,
                    'max_steps': None
                }
                
                if "ts_from_ts" in test_name:
                    test_info['starting_point'] = "TS"
                elif "ts_from_perturbed_ts" in test_name:
                    test_info['starting_point'] = "Perturbed TS"
                elif "ts_from_r_p_geo" in test_name:
                    test_info['starting_point'] = "R-P Geodesic"
                elif "ts_from_r_p" in test_name:
                    test_info['starting_point'] = "R-P Linear"
                elif "ts_from_r" in test_name:
                    if "s10000" in test_name:
                        test_info['starting_point'] = "R (10k steps)"
                        test_info['max_steps'] = 10000
                    else:
                        test_info['starting_point'] = "R"
                
                # Extract dt and steps from test name
                if "dt0.01" in test_name:
                    test_info['dt'] = 0.01
                elif "dt0.1" in test_name:
                    test_info['dt'] = 0.1
                
                if "s1000" in test_name and "s10000" not in test_name:
                    test_info['max_steps'] = 1000
                elif "s2000" in test_name:
                    test_info['max_steps'] = 2000
                
                row = {
                    'source': 'results',
                    'file_path': file_path,
                    'test_name': test_name,
                    'rmsd_final': rmsd_value,
                    **file_info,
                    **test_info
                }
                
                data.append(row)
                
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    return data

# Load all data
summary_files, results_files = find_gad_files(plots_dir, logs_dir)
if len(summary_files) == 0 and len(results_files) == 0:
    print(f"No GAD result files found in {plots_dir} or {logs_dir}")
    print("Make sure you've run the GAD tests first with --do-gad flag")
else:
    data = load_gad_data(summary_files, results_files)
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} GAD runs into DataFrame")


In [ ]:
if len(summary_files) > 0 or len(results_files) > 0:
    print("DataFrame shape:", df.shape)
    print("\nColumns:", list(df.columns))
    
    print("\nData sources:", df['source'].value_counts())
    print("\nStarting points:", df['starting_point'].value_counts())
    print("\nEigen methods:", df['eigen_method'].value_counts())
    
    # Check for key metrics
    key_metrics = ['rmsd_final', 'rmsd_initial', 'rmsd_improvement', 'nsteps', 'time_taken', 'dt', 'max_steps']
    available_metrics = [col for col in key_metrics if col in df.columns]
    print(f"\nAvailable metrics: {available_metrics}")
    
    # Show first few rows
    print("\nFirst few rows:")
    display_cols = ['source', 'starting_point', 'eigen_method', 'dt', 'max_steps'] + [col for col in ['rmsd_final', 'nsteps', 'time_taken'] if col in df.columns]
    display_cols = [col for col in display_cols if col in df.columns]
    print(df[display_cols].head(10))


In [ ]:
if len(summary_files) > 0 or len(results_files) > 0:
    # Clean the data
    df_clean = df.copy()
    
    # Handle missing values in key columns
    if 'rmsd_final' in df_clean.columns:
        print(f"Rows with missing rmsd_final: {df_clean['rmsd_final'].isna().sum()}")
        
    # Create a combined identifier for eigen method and starting point
    df_clean['eigen_start'] = df_clean['eigen_method'].astype(str) + " + " + df_clean['starting_point'].astype(str)
    
    # Create a method identifier that includes integration parameters when available
    df_clean['method_full'] = df_clean['eigen_method'].astype(str)
    
    # Add dt and max_steps info when available
    dt_mask = df_clean['dt'].notna()
    steps_mask = df_clean['max_steps'].notna()
    
    df_clean.loc[dt_mask, 'method_full'] = df_clean.loc[dt_mask, 'method_full'] + " (dt=" + df_clean.loc[dt_mask, 'dt'].astype(str) + ")"
    df_clean.loc[steps_mask, 'method_full'] = df_clean.loc[steps_mask, 'method_full'] + " (steps=" + df_clean.loc[steps_mask, 'max_steps'].astype(str) + ")"
    
    print(f"\nCleaned DataFrame shape: {df_clean.shape}")
    print(f"\nEigen + Starting point combinations:")
    print(df_clean['eigen_start'].value_counts().head(10))
    
    print(f"\nFull method combinations (top 15):")
    print(df_clean['method_full'].value_counts().head(15))


In [ ]:
if (len(summary_files) > 0 or len(results_files) > 0) and 'rmsd_final' in df_clean.columns:
    # Filter out rows with missing rmsd_final
    df_plot = df_clean.dropna(subset=['rmsd_final', 'starting_point', 'eigen_method'])
    
    if len(df_plot) == 0:
        print("No valid data for plotting")
    else:
        # Create the main plot
        fig, axes = plt.subplots(2, 2, figsize=(20, 15))
        fig.suptitle('GAD Transition State Search Results: Final RMSD Analysis', fontsize=16, fontweight='bold')
        
        # Plot 1: Box plot by starting point and eigen method
        ax1 = axes[0, 0]
        sns.boxplot(data=df_plot, x='starting_point', y='rmsd_final', hue='eigen_method', ax=ax1)
        ax1.set_title('Final RMSD by Starting Point and Eigen Method')
        ax1.set_ylabel('Final RMSD (Å)')
        ax1.tick_params(axis='x', rotation=45)
        ax1.legend(title='Eigen Method', bbox_to_anchor=(1.05, 1), loc='upper left')
        
        # Plot 2: Box plot by eigen method only
        ax2 = axes[0, 1]
        sns.boxplot(data=df_plot, x='eigen_method', y='rmsd_final', ax=ax2)
        ax2.set_title('Final RMSD by Eigen Method')
        ax2.set_ylabel('Final RMSD (Å)')
        ax2.tick_params(axis='x', rotation=45)
        
        # Plot 3: Violin plot showing distribution
        ax3 = axes[1, 0]
        if len(df_plot['starting_point'].unique()) <= 10:  # Only if not too many categories
            sns.violinplot(data=df_plot, x='starting_point', y='rmsd_final', ax=ax3)
            ax3.set_title('Final RMSD Distribution by Starting Point')
            ax3.set_ylabel('Final RMSD (Å)')
            ax3.tick_params(axis='x', rotation=45)
        else:
            ax3.text(0.5, 0.5, 'Too many starting point categories for violin plot', 
                    ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('Starting Point Categories Too Numerous')
        
        # Plot 4: Scatter plot of performance (if nsteps available)
        ax4 = axes[1, 1]
        if 'nsteps' in df_plot.columns and df_plot['nsteps'].notna().sum() > 0:
            scatter = sns.scatterplot(data=df_plot, x='nsteps', y='rmsd_final', 
                                    hue='starting_point', style='eigen_method', 
                                    s=100, ax=ax4)
            ax4.set_title('Final RMSD vs Number of Integration Steps')
            ax4.set_xlabel('Number of Integration Steps')
            ax4.set_ylabel('Final RMSD (Å)')
            ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        else:
            # Alternative plot with dt vs rmsd if available
            if 'dt' in df_plot.columns and df_plot['dt'].notna().sum() > 0:
                sns.scatterplot(data=df_plot, x='dt', y='rmsd_final', 
                              hue='starting_point', style='eigen_method', 
                              s=100, ax=ax4)
                ax4.set_title('Final RMSD vs Integration Time Step')
                ax4.set_xlabel('Time Step (dt)')
                ax4.set_ylabel('Final RMSD (Å)')
                ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            else:
                ax4.text(0.5, 0.5, 'Integration step data not available', 
                        ha='center', va='center', transform=ax4.transAxes)
                ax4.set_title('Integration Data Not Available')
        
        plt.tight_layout()
        plt.show()
        
        # Print summary statistics
        print("\n" + "="*80)
        print("GAD SUMMARY STATISTICS")
        print("="*80)
        
        print("\nFinal RMSD by Starting Point:")
        print(df_plot.groupby('starting_point')['rmsd_final'].describe())
        
        print("\nFinal RMSD by Eigen Method:")
        print(df_plot.groupby('eigen_method')['rmsd_final'].describe())
        
        # Best performing configurations
        print("\nBest performing configurations (lowest final RMSD):")
        best_cols = ['starting_point', 'eigen_method', 'dt', 'max_steps', 'rmsd_final']
        if 'nsteps' in df_plot.columns:
            best_cols.append('nsteps')
        best_cols = [col for col in best_cols if col in df_plot.columns]
        
        best_configs = df_plot.nsmallest(15, 'rmsd_final')[best_cols]
        print(best_configs.to_string(index=False))
        
        # Worst performing configurations
        print("\nWorst performing configurations (highest final RMSD):")
        worst_configs = df_plot.nlargest(10, 'rmsd_final')[best_cols]
        print(worst_configs.to_string(index=False))
        
else:
    print("No data available for plotting. Make sure GAD runs have been completed.")


In [ ]:
if (len(summary_files) > 0 or len(results_files) > 0) and 'rmsd_final' in df_clean.columns:
    df_eigen = df_clean.dropna(subset=['rmsd_final', 'eigen_method'])
    
    if len(df_eigen) > 0:
        # Detailed eigen method analysis
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle('GAD Eigen Method Detailed Analysis', fontsize=16, fontweight='bold')
        
        # Plot 1: Box plot of RMSD by eigen method
        ax1 = axes[0, 0]
        eigen_order = df_eigen.groupby('eigen_method')['rmsd_final'].median().sort_values().index
        sns.boxplot(data=df_eigen, x='eigen_method', y='rmsd_final', order=eigen_order, ax=ax1)
        ax1.set_title('Final RMSD by Eigen Method (sorted by median)')
        ax1.set_ylabel('Final RMSD (Å)')
        ax1.tick_params(axis='x', rotation=45)
        
        # Plot 2: Success rate (RMSD < threshold)
        ax2 = axes[0, 1]
        threshold = 0.1  # 0.1 Å threshold for "success"
        success_rate = df_eigen.groupby('eigen_method').apply(
            lambda x: (x['rmsd_final'] < threshold).mean() * 100
        ).sort_values(ascending=False)
        
        success_rate.plot(kind='bar', ax=ax2)
        ax2.set_title(f'Success Rate by Eigen Method (RMSD < {threshold} Å)')
        ax2.set_ylabel('Success Rate (%)')
        ax2.tick_params(axis='x', rotation=45)
        
        # Plot 3: Performance by starting point for each eigen method
        ax3 = axes[1, 0]
        if len(df_eigen['starting_point'].unique()) <= 8:  # Only if manageable number
            pivot_data = df_eigen.pivot_table(values='rmsd_final', 
                                             index='eigen_method', 
                                             columns='starting_point', 
                                             aggfunc='mean')
            sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='RdYlBu_r', ax=ax3)
            ax3.set_title('Mean Final RMSD Heatmap')
            ax3.set_xlabel('Starting Point')
            ax3.set_ylabel('Eigen Method')
        else:
            ax3.text(0.5, 0.5, 'Too many starting points for heatmap', 
                    ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('Too Many Categories for Heatmap')
        
        # Plot 4: Distribution comparison
        ax4 = axes[1, 1]
        top_methods = df_eigen['eigen_method'].value_counts().head(6).index
        df_top = df_eigen[df_eigen['eigen_method'].isin(top_methods)]
        
        for method in top_methods:
            method_data = df_top[df_top['eigen_method'] == method]['rmsd_final']
            ax4.hist(method_data, alpha=0.6, label=method, bins=20)
        
        ax4.set_title('RMSD Distribution by Top Eigen Methods')
        ax4.set_xlabel('Final RMSD (Å)')
        ax4.set_ylabel('Frequency')
        ax4.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Print statistical summary
        print("\n" + "="*60)
        print("EIGEN METHOD STATISTICAL ANALYSIS")
        print("="*60)
        
        print(f"\nSuccess rates (RMSD < {threshold} Å):")
        for method, rate in success_rate.items():
            print(f"{method:12s}: {rate:6.1f}%")
        
        print("\nMedian RMSD by eigen method:")
        median_rmsd = df_eigen.groupby('eigen_method')['rmsd_final'].median().sort_values()
        for method, rmsd in median_rmsd.items():
            print(f"{method:12s}: {rmsd:7.4f} Å")
            
    else:
        print("No valid eigen method data for analysis")


In [ ]:
if (len(summary_files) > 0 or len(results_files) > 0):
    # Analyze integration parameters (dt, max_steps) if available
    param_cols = ['dt', 'max_steps']
    available_params = [col for col in param_cols if col in df_clean.columns and df_clean[col].notna().sum() > 0]
    
    if available_params and 'rmsd_final' in df_clean.columns:
        df_params = df_clean.dropna(subset=['rmsd_final'] + available_params)
        
        if len(df_params) > 0:
            n_params = len(available_params)
            fig, axes = plt.subplots(1, n_params, figsize=(8*n_params, 6))
            if n_params == 1:
                axes = [axes]
            
            fig.suptitle('GAD Integration Parameter Analysis', fontsize=16, fontweight='bold')
            
            for i, param in enumerate(available_params):
                df_param = df_params.dropna(subset=[param])
                
                if len(df_param) > 0:
                    if param == 'dt':
                        # For dt, use scatter plot
                        sns.scatterplot(data=df_param, x=param, y='rmsd_final', 
                                      hue='starting_point', style='eigen_method',
                                      s=100, ax=axes[i])
                        axes[i].set_title(f'Final RMSD vs {param}')
                        axes[i].set_xlabel(f'{param}')
                        axes[i].set_ylabel('Final RMSD (Å)')
                    else:
                        # For max_steps, use box plot
                        sns.boxplot(data=df_param, x=param, y='rmsd_final', ax=axes[i])
                        axes[i].set_title(f'Final RMSD vs {param}')
                        axes[i].set_xlabel(f'{param}')
                        axes[i].set_ylabel('Final RMSD (Å)')
                        axes[i].tick_params(axis='x', rotation=45)
                    
                    if i > 0:  # Remove legend from subsequent plots
                        legend = axes[i].get_legend()
                        if legend:
                            legend.remove()
            
            plt.tight_layout()
            plt.show()
            
            # Print parameter analysis
            print("\n" + "="*60)
            print("INTEGRATION PARAMETER ANALYSIS")
            print("="*60)
            
            for param in available_params:
                df_param = df_params.dropna(subset=[param])
                if len(df_param) > 0:
                    print(f"\nFinal RMSD by {param}:")
                    print(df_param.groupby(param)['rmsd_final'].describe())
    
    # Time analysis if available
    if 'time_taken' in df_clean.columns and df_clean['time_taken'].notna().sum() > 0:
        df_time = df_clean.dropna(subset=['time_taken', 'rmsd_final'])
        
        if len(df_time) > 0:
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            fig.suptitle('GAD Computational Performance Analysis', fontsize=16, fontweight='bold')
            
            # Time vs RMSD
            sns.scatterplot(data=df_time, x='time_taken', y='rmsd_final', 
                          hue='eigen_method', style='starting_point', 
                          s=100, ax=axes[0])
            axes[0].set_title('Final RMSD vs Computation Time')
            axes[0].set_xlabel('Time Taken (s)')
            axes[0].set_ylabel('Final RMSD (Å)')
            
            # Time by eigen method
            sns.boxplot(data=df_time, x='eigen_method', y='time_taken', ax=axes[1])
            axes[1].set_title('Computation Time by Eigen Method')
            axes[1].set_xlabel('Eigen Method')
            axes[1].set_ylabel('Time Taken (s)')
            axes[1].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()
            
            print("\nComputation time by eigen method:")
            print(df_time.groupby('eigen_method')['time_taken'].describe())


In [ ]:
# Uncomment to save results
# if len(summary_files) > 0 or len(results_files) > 0:
#     # Save the cleaned dataframe
#     output_file = "gad_results_analysis.csv"
#     df_clean.to_csv(output_file, index=False)
#     print(f"Saved cleaned results to {output_file}")
    
#     # Save summary statistics
#     summary_stats = {
#         'total_runs': len(df_clean),
#         'successful_runs': len(df_clean.dropna(subset=['rmsd_final'])),
#         'starting_points': df_clean['starting_point'].value_counts().to_dict(),
#         'eigen_methods': df_clean['eigen_method'].value_counts().to_dict(),
#         'data_sources': df_clean['source'].value_counts().to_dict()
#     }
    
#     with open('gad_results_summary.json', 'w') as f:
#         json.dump(summary_stats, f, indent=2)
    
#     print(f"Saved summary statistics to gad_results_summary.json")
#     print(f"\nAnalysis complete! Processed {len(df_clean)} GAD optimization runs.")
# else:
#     print("No data to export. Run GAD optimization first.")

print("\n" + "="*80)
print("GAD RESULTS ANALYSIS COMPLETE")
print("="*80)
if len(summary_files) > 0 or len(results_files) > 0:
    print(f"Successfully analyzed {len(df)} GAD transition state search results.")
    print("\nKey findings:")
    if 'rmsd_final' in df_clean.columns:
        df_valid = df_clean.dropna(subset=['rmsd_final'])
        if len(df_valid) > 0:
            print(f"- Total valid runs: {len(df_valid)}")
            print(f"- Best RMSD achieved: {df_valid['rmsd_final'].min():.4f} Å")
            print(f"- Median RMSD: {df_valid['rmsd_final'].median():.4f} Å")
            if 'eigen_method' in df_valid.columns:
                best_method = df_valid.loc[df_valid['rmsd_final'].idxmin(), 'eigen_method']
                print(f"- Best performing eigen method: {best_method}")
else:
    print("No GAD results found. Please run GAD tests first with --do-gad flag.")


In [ ]:
if (len(summary_files) > 0 or len(results_files) > 0) and 'rmsd_final' in df_clean.columns:
    df_analysis = df_clean.dropna(subset=['rmsd_final', 'eigen_method'])
    
    if len(df_analysis) > 0:
        # PRIMARY ANALYSIS PLOTS
        fig, axes = plt.subplots(2, 2, figsize=(20, 15))
        fig.suptitle('GAD Primary Analysis: Core Performance Metrics', fontsize=18, fontweight='bold')
        
        # 1. Eigen Method Performance (sorted by median)
        ax1 = axes[0, 0]
        eigen_order = df_analysis.groupby('eigen_method')['rmsd_final'].median().sort_values().index
        sns.boxplot(data=df_analysis, x='eigen_method', y='rmsd_final', order=eigen_order, ax=ax1)
        ax1.set_title('1. RMSD by Eigen Method (sorted by median performance)', fontweight='bold')
        ax1.set_ylabel('Final RMSD (Å)')
        ax1.tick_params(axis='x', rotation=45)
        
        # Add median values as text
        medians = df_analysis.groupby('eigen_method')['rmsd_final'].median()
        for i, method in enumerate(eigen_order):
            ax1.text(i, medians[method] + 0.01, f'{medians[method]:.3f}', 
                    ha='center', va='bottom', fontweight='bold')
        
        # 2. Success Rate by Eigen Method
        ax2 = axes[0, 1]
        threshold = 0.1  # Success threshold
        success_rate = df_analysis.groupby('eigen_method').apply(
            lambda x: (x['rmsd_final'] < threshold).mean() * 100
        ).sort_values(ascending=False)
        
        success_rate.plot(kind='bar', ax=ax2, color='skyblue', alpha=0.8)
        ax2.set_title(f'2. Success Rate by Eigen Method (RMSD < {threshold} Å)', fontweight='bold')
        ax2.set_ylabel('Success Rate (%)')
        ax2.tick_params(axis='x', rotation=45)
        ax2.set_ylim(0, 100)
        
        # Add percentage labels on bars
        for i, (method, rate) in enumerate(success_rate.items()):
            ax2.text(i, rate + 1, f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        # 3. Starting Point Reliability
        ax3 = axes[1, 0]
        if 'starting_point' in df_analysis.columns and df_analysis['starting_point'].notna().sum() > 0:
            df_start = df_analysis.dropna(subset=['starting_point'])
            sns.boxplot(data=df_start, x='starting_point', y='rmsd_final', hue='eigen_method', ax=ax3)
            ax3.set_title('3. RMSD by Starting Point and Eigen Method', fontweight='bold')
            ax3.set_ylabel('Final RMSD (Å)')
            ax3.tick_params(axis='x', rotation=45)
            ax3.legend(title='Eigen Method', bbox_to_anchor=(1.05, 1), loc='upper left')
        else:
            ax3.text(0.5, 0.5, 'Starting point data not available', 
                    ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('3. Starting Point Analysis - Data Not Available')
        
        # 4. Distribution Comparison (Top Methods)
        ax4 = axes[1, 1]
        top_methods = df_analysis['eigen_method'].value_counts().head(5).index
        colors = plt.cm.Set3(np.linspace(0, 1, len(top_methods)))
        
        for i, method in enumerate(top_methods):
            method_data = df_analysis[df_analysis['eigen_method'] == method]['rmsd_final']
            if len(method_data) > 0:
                ax4.hist(method_data, alpha=0.7, label=f'{method} (n={len(method_data)})', 
                        bins=15, color=colors[i])
        
        ax4.set_title('4. RMSD Distribution by Top Eigen Methods', fontweight='bold')
        ax4.set_xlabel('Final RMSD (Å)')
        ax4.set_ylabel('Frequency')
        ax4.legend()
        ax4.axvline(x=threshold, color='red', linestyle='--', alpha=0.7, 
                   label=f'Success threshold ({threshold} Å)')
        
        plt.tight_layout()
        plt.show()
        
        # Print key insights
        print("\n" + "="*80)
        print("KEY INSIGHTS FROM PRIMARY ANALYSIS")
        print("="*80)
        
        print(f"\n📊 BEST PERFORMING EIGEN METHODS (by median RMSD):")
        for i, (method, median_rmsd) in enumerate(medians.sort_values().head(3).items()):
            success_pct = success_rate[method] if method in success_rate.index else 0
            print(f"  {i+1}. {method:12s}: {median_rmsd:.4f} Å (success: {success_pct:.1f}%)")
        
        print(f"\n🎯 MOST RELIABLE METHODS (by success rate):")
        for i, (method, rate) in enumerate(success_rate.head(3).items()):
            median_rmsd = medians[method] if method in medians.index else float('inf')
            print(f"  {i+1}. {method:12s}: {rate:.1f}% success (median: {median_rmsd:.4f} Å)")
        
        print(f"\n📈 PERFORMANCE SUMMARY:")
        print(f"  • Total runs analyzed: {len(df_analysis)}")
        print(f"  • Overall success rate: {(df_analysis['rmsd_final'] < threshold).mean()*100:.1f}%")
        print(f"  • Best RMSD achieved: {df_analysis['rmsd_final'].min():.4f} Å")
        print(f"  • Median RMSD overall: {df_analysis['rmsd_final'].median():.4f} Å")
    
    else:
        print("No valid data for primary analysis plots")


In [ ]:
if (len(summary_files) > 0 or len(results_files) > 0) and 'rmsd_final' in df_clean.columns:
    df_secondary = df_clean.dropna(subset=['rmsd_final'])
    
    if len(df_secondary) > 0:
        # SECONDARY ANALYSIS PLOTS
        fig, axes = plt.subplots(2, 2, figsize=(22, 16))
        fig.suptitle('GAD Secondary Analysis: Interaction Effects & Optimization', fontsize=18, fontweight='bold')
        
        # 1. Method × Starting Point Heatmap
        ax1 = axes[0, 0]
        if ('eigen_method' in df_secondary.columns and 'starting_point' in df_secondary.columns and 
            df_secondary[['eigen_method', 'starting_point']].notna().all(axis=1).sum() > 0):
            
            df_heatmap = df_secondary.dropna(subset=['eigen_method', 'starting_point'])
            
            # Only create heatmap if we have reasonable number of categories
            n_eigen = df_heatmap['eigen_method'].nunique()
            n_start = df_heatmap['starting_point'].nunique()
            
            if n_eigen <= 12 and n_start <= 10:  # Reasonable limits for heatmap
                pivot_table = df_heatmap.pivot_table(values='rmsd_final', 
                                                    index='eigen_method', 
                                                    columns='starting_point', 
                                                    aggfunc='mean')
                
                # Create heatmap with custom colormap
                sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='RdYlBu_r', 
                           ax=ax1, cbar_kws={'label': 'Mean RMSD (Å)'})
                ax1.set_title('1. Method × Starting Point Interaction', fontweight='bold')
                ax1.set_xlabel('Starting Point')
                ax1.set_ylabel('Eigen Method')
            else:
                ax1.text(0.5, 0.5, f'Too many categories for heatmap\\n({n_eigen} methods × {n_start} starts)', 
                        ha='center', va='center', transform=ax1.transAxes)
                ax1.set_title('1. Method × Starting Point - Too Many Categories')
        else:
            ax1.text(0.5, 0.5, 'Insufficient interaction data', 
                    ha='center', va='center', transform=ax1.transAxes)
            ax1.set_title('1. Method × Starting Point - Data Not Available')
        
        # 2. Integration Parameter Effects
        ax2 = axes[0, 1]
        param_cols = ['dt', 'max_steps']
        available_params = [col for col in param_cols if col in df_secondary.columns and 
                           df_secondary[col].notna().sum() > 5]  # At least 5 data points
        
        if available_params:
            # Focus on dt if available, otherwise max_steps
            param = 'dt' if 'dt' in available_params else available_params[0]
            df_param = df_secondary.dropna(subset=[param, 'eigen_method'])
            
            if len(df_param) > 0:
                if param == 'dt':
                    # Scatter plot for continuous dt values
                    sns.scatterplot(data=df_param, x=param, y='rmsd_final', 
                                  hue='eigen_method', style='starting_point', 
                                  s=100, ax=ax2, alpha=0.8)
                    ax2.set_xlabel('Integration Time Step (dt)')
                else:
                    # Box plot for discrete max_steps values
                    sns.boxplot(data=df_param, x=param, y='rmsd_final', ax=ax2)
                    ax2.set_xlabel('Max Steps')
                    ax2.tick_params(axis='x', rotation=45)
                
                ax2.set_title(f'2. Integration Parameter Effects ({param})', fontweight='bold')
                ax2.set_ylabel('Final RMSD (Å)')
                if param == 'dt':
                    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            else:
                ax2.text(0.5, 0.5, f'No valid {param} data', 
                        ha='center', va='center', transform=ax2.transAxes)
                ax2.set_title(f'2. Parameter Effects - No {param} Data')
        else:
            ax2.text(0.5, 0.5, 'No integration parameter data', 
                    ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('2. Integration Parameter Effects - Data Not Available')
        
        # 3. Efficiency vs Quality Trade-off
        ax3 = axes[1, 0]
        if ('time_taken' in df_secondary.columns and 
            df_secondary['time_taken'].notna().sum() > 5):
            
            df_efficiency = df_secondary.dropna(subset=['time_taken', 'eigen_method'])
            
            # Create efficiency scatter plot
            scatter = sns.scatterplot(data=df_efficiency, x='time_taken', y='rmsd_final', 
                                    hue='eigen_method', style='starting_point',
                                    s=120, ax=ax3, alpha=0.8)
            
            ax3.set_title('3. Efficiency vs Quality Trade-off', fontweight='bold')
            ax3.set_xlabel('Computation Time (s)')
            ax3.set_ylabel('Final RMSD (Å)')
            
            # Add Pareto frontier suggestion
            ax3.axhline(y=0.1, color='red', linestyle='--', alpha=0.5, label='Success threshold')
            ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            
            # Identify and highlight Pareto optimal points
            # (Points that are both fast and accurate)
            median_time = df_efficiency['time_taken'].median()
            fast_and_accurate = df_efficiency[
                (df_efficiency['time_taken'] <= median_time) & 
                (df_efficiency['rmsd_final'] <= 0.1)
            ]
            
            if len(fast_and_accurate) > 0:
                ax3.scatter(fast_and_accurate['time_taken'], fast_and_accurate['rmsd_final'], 
                           s=200, facecolors='none', edgecolors='green', linewidth=3,
                           label='Fast & Accurate')
        else:
            ax3.text(0.5, 0.5, 'No timing data available', 
                    ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('3. Efficiency Analysis - No Timing Data')
        
        # 4. Robustness Analysis (Consistency)
        ax4 = axes[1, 1]
        if len(df_secondary) > 0:
            # Calculate coefficient of variation (std/mean) for each eigen method
            consistency_stats = df_secondary.groupby('eigen_method')['rmsd_final'].agg([
                'mean', 'std', 'count'
            ]).reset_index()
            
            # Filter methods with enough data points
            consistency_stats = consistency_stats[consistency_stats['count'] >= 3]
            
            if len(consistency_stats) > 0:
                consistency_stats['cv'] = consistency_stats['std'] / consistency_stats['mean']
                consistency_stats = consistency_stats.sort_values('cv')
                
                # Create consistency plot
                bars = ax4.bar(range(len(consistency_stats)), consistency_stats['cv'], 
                              color='lightcoral', alpha=0.8)
                ax4.set_xticks(range(len(consistency_stats)))
                ax4.set_xticklabels(consistency_stats['eigen_method'], rotation=45)
                ax4.set_title('4. Method Robustness (Lower = More Consistent)', fontweight='bold')
                ax4.set_ylabel('Coefficient of Variation (std/mean)')
                
                # Add value labels on bars
                for i, bar in enumerate(bars):
                    height = bar.get_height()
                    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')
            else:
                ax4.text(0.5, 0.5, 'Insufficient data for consistency analysis', 
                        ha='center', va='center', transform=ax4.transAxes)
                ax4.set_title('4. Robustness Analysis - Insufficient Data')
        
        plt.tight_layout()
        plt.show()
        
        # Print secondary insights
        print("\n" + "="*80)
        print("SECONDARY ANALYSIS INSIGHTS")
        print("="*80)
        
        # Interaction effects
        if ('eigen_method' in df_secondary.columns and 'starting_point' in df_secondary.columns):
            df_interact = df_secondary.dropna(subset=['eigen_method', 'starting_point'])
            if len(df_interact) > 0:
                print(f"\n🔄 INTERACTION EFFECTS:")
                best_combos = df_interact.groupby(['eigen_method', 'starting_point'])['rmsd_final'].mean().nsmallest(5)
                for i, ((method, start), rmsd) in enumerate(best_combos.items()):
                    print(f"  {i+1}. {method:10s} + {start:15s}: {rmsd:.4f} Å")
        
        # Parameter optimization
        if available_params:
            param = 'dt' if 'dt' in available_params else available_params[0]
            df_param_opt = df_secondary.dropna(subset=[param])
            if len(df_param_opt) > 0:
                print(f"\n⚙️  PARAMETER OPTIMIZATION ({param.upper()}):")
                param_performance = df_param_opt.groupby(param)['rmsd_final'].agg(['mean', 'count'])
                param_performance = param_performance[param_performance['count'] >= 2]  # At least 2 samples
                if len(param_performance) > 0:
                    best_param = param_performance['mean'].idxmin()
                    print(f"  • Best {param}: {best_param} (mean RMSD: {param_performance.loc[best_param, 'mean']:.4f} Å)")
        
        # Efficiency insights
        if 'time_taken' in df_secondary.columns:
            df_timing = df_secondary.dropna(subset=['time_taken', 'eigen_method'])
            if len(df_timing) > 0:
                print(f"\n⚡ EFFICIENCY INSIGHTS:")
                efficiency_stats = df_timing.groupby('eigen_method').agg({
                    'time_taken': 'mean',
                    'rmsd_final': ['mean', lambda x: (x < 0.1).mean() * 100]
                }).round(3)
                efficiency_stats.columns = ['mean_time', 'mean_rmsd', 'success_rate']
                efficiency_stats['efficiency_score'] = efficiency_stats['success_rate'] / efficiency_stats['mean_time']
                
                top_efficient = efficiency_stats.nlargest(3, 'efficiency_score')
                for i, (method, stats) in enumerate(top_efficient.iterrows()):
                    print(f"  {i+1}. {method:12s}: {stats['efficiency_score']:.3f} (success/time ratio)")
    
    else:
        print("No valid data for secondary analysis plots")


In [ ]:
if (len(summary_files) > 0 or len(results_files) > 0) and 'rmsd_final' in df_clean.columns:
    df_advanced = df_clean.dropna(subset=['rmsd_final', 'eigen_method'])
    
    if len(df_advanced) > 0:
        # ADVANCED ANALYSIS: RECOMMENDATION SYSTEM
        fig, axes = plt.subplots(2, 2, figsize=(20, 14))
        fig.suptitle('GAD Advanced Analysis: Practical Decision Making', fontsize=18, fontweight='bold')
        
        # Calculate recommendation scores for each eigen method
        method_stats = df_advanced.groupby('eigen_method').agg({
            'rmsd_final': ['mean', 'median', 'std', 'count', lambda x: (x < 0.1).mean()]
        }).round(4)
        
        method_stats.columns = ['mean_rmsd', 'median_rmsd', 'std_rmsd', 'count', 'success_rate']
        method_stats = method_stats[method_stats['count'] >= 2]  # At least 2 samples
        
        if len(method_stats) > 0:
            # 1. Recommendation Score Calculation
            ax1 = axes[0, 0]
            
            # Normalize metrics (lower RMSD and higher success rate are better)
            method_stats['norm_median'] = 1 - (method_stats['median_rmsd'] - method_stats['median_rmsd'].min()) / (method_stats['median_rmsd'].max() - method_stats['median_rmsd'].min() + 1e-8)
            method_stats['norm_success'] = method_stats['success_rate']
            method_stats['norm_consistency'] = 1 - (method_stats['std_rmsd'] - method_stats['std_rmsd'].min()) / (method_stats['std_rmsd'].max() - method_stats['std_rmsd'].min() + 1e-8)
            
            # Add timing if available
            if 'time_taken' in df_advanced.columns:
                timing_stats = df_advanced.groupby('eigen_method')['time_taken'].mean()
                method_stats = method_stats.join(timing_stats, how='left')
                method_stats['time_taken'] = method_stats['time_taken'].fillna(method_stats['time_taken'].median())
                method_stats['norm_speed'] = 1 - (method_stats['time_taken'] - method_stats['time_taken'].min()) / (method_stats['time_taken'].max() - method_stats['time_taken'].min() + 1e-8)
                
                # Weighted recommendation score
                method_stats['recommendation_score'] = (
                    0.35 * method_stats['norm_success'] +      # 35% success rate
                    0.25 * method_stats['norm_median'] +       # 25% median performance  
                    0.20 * method_stats['norm_speed'] +        # 20% speed
                    0.20 * method_stats['norm_consistency']     # 20% consistency
                )
            else:
                # Without timing data
                method_stats['recommendation_score'] = (
                    0.40 * method_stats['norm_success'] +      # 40% success rate
                    0.35 * method_stats['norm_median'] +       # 35% median performance
                    0.25 * method_stats['norm_consistency']     # 25% consistency
                )
            
            # Plot recommendation scores
            sorted_methods = method_stats.sort_values('recommendation_score', ascending=True)
            bars = ax1.barh(range(len(sorted_methods)), sorted_methods['recommendation_score'], 
                           color='lightgreen', alpha=0.8)
            ax1.set_yticks(range(len(sorted_methods)))
            ax1.set_yticklabels(sorted_methods.index)
            ax1.set_title('1. Overall Recommendation Score', fontweight='bold')
            ax1.set_xlabel('Recommendation Score (Higher = Better)')
            
            # Add score labels
            for i, bar in enumerate(bars):
                width = bar.get_width()
                ax1.text(width + 0.01, bar.get_y() + bar.get_height()/2,
                        f'{width:.3f}', ha='left', va='center', fontweight='bold')
        
        # 2. Performance Radar Chart (for top 3 methods)
        ax2 = axes[0, 1]
        if len(method_stats) >= 3:
            top_3_methods = method_stats.nlargest(3, 'recommendation_score')
            
            # Radar chart data
            categories = ['Success\\nRate', 'Accuracy\\n(low RMSD)', 'Consistency', 'Speed']
            if 'time_taken' not in df_advanced.columns:
                categories = categories[:-1]  # Remove speed if no timing data
            
            # Create radar chart
            angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
            angles += angles[:1]  # Complete the circle
            
            colors = ['red', 'blue', 'green']
            for i, (method, stats) in enumerate(top_3_methods.iterrows()):
                values = [stats['norm_success'], stats['norm_median'], stats['norm_consistency']]
                if 'norm_speed' in stats:
                    values.append(stats['norm_speed'])
                values += values[:1]  # Complete the circle
                
                ax2.plot(angles, values, 'o-', linewidth=2, label=method, color=colors[i], alpha=0.8)
                ax2.fill(angles, values, alpha=0.1, color=colors[i])
            
            ax2.set_xticks(angles[:-1])
            ax2.set_xticklabels(categories)
            ax2.set_ylim(0, 1)
            ax2.set_title('2. Top 3 Methods Performance Profile', fontweight='bold')
            ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
            ax2.grid(True)
        else:
            ax2.text(0.5, 0.5, 'Need at least 3 methods for radar chart', 
                    ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('2. Performance Profile - Insufficient Methods')
        
        # 3. Decision Matrix
        ax3 = axes[1, 0]
        if len(method_stats) > 0:
            # Create decision matrix heatmap
            decision_cols = ['success_rate', 'median_rmsd', 'std_rmsd']
            if 'time_taken' in method_stats.columns:
                decision_cols.append('time_taken')
            
            decision_matrix = method_stats[decision_cols].copy()
            decision_matrix.columns = ['Success Rate', 'Median RMSD', 'Std RMSD', 'Mean Time'] if 'time_taken' in method_stats.columns else ['Success Rate', 'Median RMSD', 'Std RMSD']
            
            # Normalize for heatmap (green = good, red = bad)
            normalized_matrix = decision_matrix.copy()
            normalized_matrix['Success Rate'] = decision_matrix['Success Rate']  # Higher is better
            normalized_matrix['Median RMSD'] = 1 - (decision_matrix['Median RMSD'] - decision_matrix['Median RMSD'].min()) / (decision_matrix['Median RMSD'].max() - decision_matrix['Median RMSD'].min() + 1e-8)  # Lower is better
            normalized_matrix['Std RMSD'] = 1 - (decision_matrix['Std RMSD'] - decision_matrix['Std RMSD'].min()) / (decision_matrix['Std RMSD'].max() - decision_matrix['Std RMSD'].min() + 1e-8)  # Lower is better
            if 'Mean Time' in normalized_matrix.columns:
                normalized_matrix['Mean Time'] = 1 - (decision_matrix['Mean Time'] - decision_matrix['Mean Time'].min()) / (decision_matrix['Mean Time'].max() - decision_matrix['Mean Time'].min() + 1e-8)  # Lower is better
            
            sns.heatmap(normalized_matrix, annot=decision_matrix, fmt='.3f', 
                       cmap='RdYlGn', ax=ax3, cbar_kws={'label': 'Normalized Score (Green = Better)'})
            ax3.set_title('3. Decision Matrix (Annotated with Raw Values)', fontweight='bold')
            ax3.set_xlabel('Performance Metrics')
            ax3.set_ylabel('Eigen Methods')
        
        # 4. Practical Recommendations
        ax4 = axes[1, 1]
        ax4.axis('off')  # Remove axes for text display
        
        # if len(method_stats) > 0:
        #     # Generate practical recommendations
        #     best_overall = method_stats.loc[method_stats['recommendation_score'].idxmax()]
        #     best_accuracy = method_stats.loc[method_stats['median_rmsd'].idxmin()]
        #     best_reliability = method_stats.loc[method_stats['success_rate'].idxmax()]
        #     most_consistent = method_stats.loc[method_stats['std_rmsd'].idxmin()]
            
        #     recommendations_text = f\"\"\"PRACTICAL RECOMMENDATIONS\n\n🏆 BEST OVERALL: {best_overall.name}\n   Score: {best_overall['recommendation_score']:.3f}\n   Success: {best_overall['success_rate']*100:.1f}%\n   Median RMSD: {best_overall['median_rmsd']:.4f} Å\n\n🎯 MOST ACCURATE: {best_accuracy.name}\n   Median RMSD: {best_accuracy['median_rmsd']:.4f} Å\n   Success: {best_accuracy['success_rate']*100:.1f}%\n\n🔒 MOST RELIABLE: {best_reliability.name}\n   Success Rate: {best_reliability['success_rate']*100:.1f}%\n   Median RMSD: {best_reliability['median_rmsd']:.4f} Å\n\n📊 MOST CONSISTENT: {most_consistent.name}\n   Std Dev: {most_consistent['std_rmsd']:.4f} Å\n   Success: {most_consistent['success_rate']*100:.1f}%\"\"\"\n            \n            if 'time_taken' in method_stats.columns:\n                fastest = method_stats.loc[method_stats['time_taken'].idxmin()]\n                recommendations_text += f\"\"\"\\n\\n⚡ FASTEST: {fastest.name}\\n   Time: {fastest['time_taken']:.2f} s\\n   Success: {fastest['success_rate']*100:.1f}%\"\"\"\n            \n            ax4.text(0.05, 0.95, recommendations_text, transform=ax4.transAxes, \n                    fontsize=11, verticalalignment='top', fontfamily='monospace',\n                    bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))\n            \n            ax4.set_title('4. Practical Recommendations', fontweight='bold', y=0.98)\n        
        plt.tight_layout()
        plt.show()
        
